In [5]:
%matplotlib inline
#!/usr/bin/env python3
"""
Driver script for fast-radial-scan analysis.
Processes one FRS at a time and prompts user between intervals.
"""
import functions as fn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import shutil

def cleanup_previous_outputs():
    """Delete any previous outputs before next FRS run."""
    # delete all per-window CSVs
    for f in glob.glob("derived_range_*.csv"):
        os.remove(f)
    # optionally also remove summary CSV
    if os.path.exists("high_sigma_low_beta_windows.csv"):
        os.remove("high_sigma_low_beta_windows.csv")
    print("🧹 Previous outputs removed.")

def main():
    time_ranges = fn.parse_time_ranges_from_file("PSP_fastRadialScans.txt")
    fields_files, spc_files = fn.download_psp_data(time_ranges)
    fields_xrs , spc_xrs    = fn.convert_all_cdf_to_xarray(fields_files, spc_files)

    frs_sec = [(fn.datetime64_to_j2000_seconds(s),
                fn.datetime64_to_j2000_seconds(e)) for s,e in time_ranges]

    for i,(fxr,sxr) in enumerate(zip(fields_xrs, spc_xrs)):
        if fxr is None or sxr is None: continue
        print(f"\n🔹 Processing range {i}: {time_ranges[i]}")

        selected = []
        cleanup_previous_outputs()  # wipe any outputs before this FRS

        B,V,n,wp,tB,tSC = fn.extract_psp_data(fxr,sxr)
        B_int,Bmag,Vmag,t = fn.process_range_wavelet(B,V,n,wp,tB,tSC)

        wins = fn.generate_valid_windows_aligned(
                   t, B_int, V, *frs_sec[i])

        for w_idx,(ws,we) in enumerate(wins):
            mask = (t>=ws)&(t<we)
            t_win,B_win,V_win,n_win = t[mask],B_int[mask],V[mask],n[mask]
            fs = 1/np.median(np.diff(t_win))

            csv_name = f"derived_range_{i:02d}_win_{w_idx:02d}.csv"
            fn.calculate_derived_parameters_to_csv(
                B_win, V_win, n_win, wp[mask], t_win,
                fn.j2000_to_iso(ws), fn.j2000_to_iso(we), csv_name)

            # PSD panel
            fB,PB = fn.trace_psd(B_win, fs)
            fV,PV = fn.trace_psd(V_win, fs)
            Zp,Zm = fn.compute_elsasser(V_win,B_win,n_win)
            fZp,PZp = fn.trace_psd(Zp,fs)
            fZm,PZm = fn.trace_psd(Zm,fs)

            plt.figure(figsize=(9,5))
            plt.loglog(fB,PB,label="Trace B")
            plt.loglog(fV,PV,label="Trace V")
            plt.loglog(fZp,PZp,label="Trace Z+")
            plt.loglog(fZm,PZm,label="Trace Z−")
            plt.title(f"Trace PSD  {fn.j2000_to_iso(ws)} – {fn.j2000_to_iso(we)}")
            plt.xlabel("f [Hz]"); plt.ylabel("PSD [(km/s)² Hz⁻¹]")
            plt.grid(True,which="both",ls="--",lw=0.4); plt.legend()
            plt.tight_layout(); plt.show()

            # Wavelet diagnostics
            try:
                fn.analyze_wavelet_for_range(
                    t_win, B_win, V_win,
                    np.linalg.norm(B_win,1),
                    np.linalg.norm(V_win,1),
                    (ws,we))
            except Exception as e:
                print("⚠️ wavelet skipped:", e)

            beta_mean = np.nanmean(0.03948*n_win*wp[mask]/np.clip(Bmag[mask]**2,1e-12,None))
            if fn.compute_sigma_c(V_win,B_win) > .5 and beta_mean < .4:
                selected.append({
                    "range_idx":i,"window_idx":w_idx,
                    "start_iso":fn.j2000_to_iso(ws),
                    "end_iso":fn.j2000_to_iso(we),
                    "beta_p":round(beta_mean,3)
                })

        if selected:
            pd.DataFrame(selected).to_csv("high_sigma_low_beta_windows.csv",index=False)
            print("✅ Wrote summary of qualifying windows.")
        else:
            print("⚠️ No windows satisfied selection criteria.")

        # ───── Ask user whether to continue ─────
        resp = input(f"➡️  Finished FRS {i}. Continue to next interval? [y/N]: ").strip().lower()
        if resp != "y":
            print("👋 Exiting after current FRS.")
            break

if __name__ == "__main__":
    main()

15-Jul-25 14:32:56: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_sc_4_per_cycle/2018/
15-Jul-25 14:32:56: File is current: psp_data/fields/l2/mag_sc_4_per_cycle/2018/psp_fld_l2_mag_sc_4_sa_per_cyc_20181102_v02.cdf
15-Jul-25 14:32:56: File is current: psp_data/fields/l2/mag_sc_4_per_cycle/2018/psp_fld_l2_mag_sc_4_sa_per_cyc_20181103_v02.cdf
15-Jul-25 14:32:56: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2018/


Using LEVEL=L3


15-Jul-25 14:32:56: File is current: psp_data/sweap/spc/l3/l3i/2018/psp_swp_spc_l3i_20181102_v01.cdf
15-Jul-25 14:32:56: File is current: psp_data/sweap/spc/l3/l3i/2018/psp_swp_spc_l3i_20181103_v01.cdf
15-Jul-25 14:32:56: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_sc_4_per_cycle/2018/
15-Jul-25 14:32:56: File is current: psp_data/fields/l2/mag_sc_4_per_cycle/2018/psp_fld_l2_mag_sc_4_sa_per_cyc_20181108_v02.cdf
15-Jul-25 14:32:56: File is current: psp_data/fields/l2/mag_sc_4_per_cycle/2018/psp_fld_l2_mag_sc_4_sa_per_cyc_20181109_v02.cdf
15-Jul-25 14:32:56: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2018/


Using LEVEL=L3


15-Jul-25 14:32:56: File is current: psp_data/sweap/spc/l3/l3i/2018/psp_swp_spc_l3i_20181108_v01.cdf
15-Jul-25 14:32:56: File is current: psp_data/sweap/spc/l3/l3i/2018/psp_swp_spc_l3i_20181109_v01.cdf
15-Jul-25 14:32:56: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_sc_4_per_cycle/2019/
15-Jul-25 14:32:57: File is current: psp_data/fields/l2/mag_sc_4_per_cycle/2019/psp_fld_l2_mag_sc_4_sa_per_cyc_20190401_v02.cdf
15-Jul-25 14:32:57: File is current: psp_data/fields/l2/mag_sc_4_per_cycle/2019/psp_fld_l2_mag_sc_4_sa_per_cyc_20190402_v02.cdf
15-Jul-25 14:32:57: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2019/


...

15-Jul-25 14:33:25: Remote index not found: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2025/
15-Jul-25 14:33:27: Skipping remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2025/ (previous attempt failed)
15-Jul-25 14:33:27: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_sc_4_per_cycle/2025/
15-Jul-25 14:33:27: Remote index not found: https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_sc_4_per_cycle/2025/
15-Jul-25 14:33:29: Skipping remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_sc_4_per_cycle/2025/ (previous attempt failed)
15-Jul-25 14:33:29: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2025/


Using LEVEL=L3


15-Jul-25 14:33:29: Remote index not found: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2025/
15-Jul-25 14:33:31: Skipping remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2025/ (previous attempt failed)
15-Jul-25 14:33:31: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_sc_4_per_cycle/2025/
15-Jul-25 14:33:31: Remote index not found: https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_sc_4_per_cycle/2025/
15-Jul-25 14:33:33: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2025/


Using LEVEL=L3


15-Jul-25 14:33:34: Remote index not found: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2025/


In [3]:
import os
print(os.getcwd())
import sys 
print(sys.path)
!ls

/home/williamrkiv/solar_wind_turbulence
['/home/williamrkiv/miniforge3/envs/cfa_reu/lib/python311.zip', '/home/williamrkiv/miniforge3/envs/cfa_reu/lib/python3.11', '/home/williamrkiv/miniforge3/envs/cfa_reu/lib/python3.11/lib-dynload', '', '/home/williamrkiv/miniforge3/envs/cfa_reu/lib/python3.11/site-packages']
acf.png
assign.ipynb
assign.py
crosshelicity.png
density_derived_theta.png
derived_parameters_range_00_window_00.csv
derived_parameters_range_00_window_01.csv
derived_parameters_range_00_window_02.csv
derived_parameters_range_00_window_03.csv
derived_parameters_range_00_window_04.csv
derived_parameters_range_00_window_05.csv
derived_parameters_range_00_window_06.csv
derived_parameters_range_00_window_07.csv
derived_parameters_range_00_window_08.csv
derived_parameters_range_00_window_09.csv
derived_parameters_range_00_window_10.csv
derived_parameters_range_00_window_11.csv
derived_parameters_range_00_window_12.csv
derived_parameters_range_00_window_13.csv
derived_parameters_rang